# Modelo CNN-Transformer para Pronóstico de Series Temporales Financieras

El pronóstico de series temporales financieras representa un desafío significativo debido a la naturaleza compleja de los patrones temporales presentes en los datos de mercados. Un enfoque innovador que ha demostrado resultados prometedores es el modelo híbrido CNN-Transformer, que combina las capacidades de las redes neuronales convolucionales para capturar patrones locales y la arquitectura Transformer para modelar dependencias de largo alcance. Este estudio profundiza en el trabajo de Zeng et al. (2023), que desarrolló un modelo CNN-Transformer para Series Temporales (CTTS) específicamente diseñado para predicción financiera.

## Introducción al Pronóstico de Series Temporales

El pronóstico de series temporales constituye un elemento fundamental en numerosos dominios, pero resulta particularmente desafiante en los mercados financieros debido a la presencia simultánea de patrones complejos a corto y largo plazo[1]. Los métodos tradicionales como ARIMA (Autoregressive Integrated Moving Average) o EMA (Exponential Moving Average) han sido ampliamente utilizados, pero presentan limitaciones significativas al intentar modelar relaciones no lineales o detectar patrones complejos en datos financieros[1]. Con el auge del aprendizaje profundo, las arquitecturas neuronales han ganado prominencia como alternativas más potentes para estos problemas complejos de predicción.

En la última década, las redes neuronales recurrentes (RNN) y especialmente las redes LSTM (Long Short-Term Memory) dominaron el campo del análisis de series temporales debido a su capacidad para retener información a lo largo de secuencias. Sin embargo, recientemente las Redes Neuronales Convolucionales (CNN), tradicionalmente asociadas con el procesamiento de imágenes, han demostrado un desempeño sorprendente en aplicaciones de series temporales[1][2]. El trabajo de Zeng et al. publicado en abril de 2023 exploró un enfoque híbrido particularmente efectivo: combinar CNNs con arquitecturas Transformer para aprovechar las fortalezas complementarias de ambos modelos[2].

### Arquitecturas Neuronales para Series Temporales

Las diferentes arquitecturas neuronales ofrecen ventajas distintas para el análisis de series temporales:

1. **Redes Recurrentes (RNN/LSTM/GRU)**: Diseñadas específicamente para datos secuenciales, mantienen una "memoria" de los valores anteriores. Las LSTM resuelven el problema del desvanecimiento del gradiente en RNNs tradicionales, permitiendo capturar dependencias a más largo plazo[3].

2. **Redes Convolucionales (CNN)**: Aunque originalmente creadas para procesamiento de imágenes, han demostrado efectividad en series temporales al detectar patrones locales mediante filtros convolucionales deslizantes[1]. Las CNN pueden identificar eficientemente características relevantes en ventanas temporales cortas.

3. **Arquitecturas Transformer**: Introducidas originalmente para procesamiento de lenguaje natural, emplean mecanismos de auto-atención que permiten modelar relaciones entre elementos distantes de una secuencia sin las limitaciones de las arquitecturas recurrentes[1].

4. **Modelos Híbridos**: Combinan diferentes arquitecturas para aprovechar las fortalezas complementarias. El modelo CNN-Transformer (CTTS) representa un ejemplo exitoso de esta estrategia[1][2].

## Metodología: Modelo CNN-Transformer (CTTS)

### Fundamentos Teóricos

El modelo CNN-Transformer Time Series (CTTS) propuesto por Zeng et al. aborda las limitaciones individuales de las CNN y los Transformers combinándolos estratégicamente[1]. El fundamento teórico se basa en que:

1. Las CNNs son excelentes para extraer características locales y patrones a corto plazo en datos secuenciales mediante operaciones de convolución.
2. Los Transformers, con su mecanismo de auto-atención, pueden modelar eficientemente dependencias a largo plazo y captar relaciones entre puntos temporales distantes.
3. La combinación de ambos enfoques permite al modelo capturar simultáneamente las dinámicas a corto y largo plazo presentes en series financieras[1].

### Arquitectura del Modelo

El modelo CTTS sigue una estructura secuencial donde primero se aplica una CNN unidimensional para extraer características locales, seguida por un Transformer para modelar dependencias a largo plazo[1]. A continuación se detalla cada componente:

1. **Capa CNN 1D**: Aplica filtros convolucionales a lo largo de la dimensión temporal para extraer características locales. La CNN desliza una ventana sobre la serie temporal, y cada ventana es procesada para producir un vector de embedding (un "token") que captura patrones a corto plazo[1].

2. **Embeddings Posicionales**: Se añaden a los tokens para mantener la información del orden temporal, ya que el Transformer no tiene una noción inherente de secuencialidad[1].

3. **Encoder Transformer**: La secuencia de tokens con embeddings posicionales se alimenta a un encoder Transformer. La capa Transformer aprende las dependencias a largo plazo entre puntos temporales distantes mediante mecanismos de auto-atención[1].

4. **Red Feed-Forward (MLP)**: La salida del Transformer se pasa a una red feed-forward con una capa softmax para predecir la tendencia futura (subida, bajada o estabilidad del precio)[1].

### Implementación en Python

A continuación se presenta una implementación conceptual del modelo CTTS utilizando PyTorch:



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length=5000):
        super(PositionalEncoding, self).__init__()
        
        # Crear una matriz de embeddings posicionales
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        # Registrar el buffer (parámetro no entrenado)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        # x tiene la forma [batch_size, seq_len, d_model]
        return x + self.pe[:, :x.size(1), :]

class CNNTransformerModel(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=8, num_layers=2, num_classes=3):
        super(CNNTransformerModel, self).__init__()
        
        # CNN para extraer características locales
        self.conv1 = nn.Conv1d(input_dim, d_model, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(d_model, d_model, kernel_size=3, padding=1)
        
        # Codificación posicional
        self.positional_encoding = PositionalEncoding(d_model)
        
        # Encoder Transformer
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)
        
        # Capa de clasificación final
        self.fc = nn.Linear(d_model, num_classes)
        
    def forward(self, x):
        # x tiene la forma [batch_size, seq_len, input_dim]
        
        # Transformar para la convolución: [batch_size, input_dim, seq_len]
        x = x.permute(0, 2, 1)
        
        # Aplicar CNN
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        
        # Volver a la forma para el Transformer: [batch_size, seq_len, d_model]
        x = x.permute(0, 2, 1)
        
        # Añadir embeddings posicionales
        x = self.positional_encoding(x)
        
        # Preparar formato para Transformer: [seq_len, batch_size, d_model]
        x = x.permute(1, 0, 2)
        
        # Aplicar Transformer
        x = self.transformer_encoder(x)
        
        # Tomar solo el último token para la predicción
        x = x[-1, :, :]
        
        # Clasificación final
        x = self.fc(x)
        
        return x

Ahora implementamos el proceso de entrenamiento y evaluación del modelo:

In [ ]:
# Hiperparámetros
BATCH_SIZE = 64
LEARNING_RATE = 0.001
EPOCHS = 100
SEQUENCE_LENGTH = 80  # 80 minutos de historia
INPUT_DIM = 1  # Solo precio de cierre, se podría ampliar a más características
D_MODEL = 64
NUM_CLASSES = 3  # Arriba, abajo, plano

# Simulación de datos para demostración
# En un caso real, se usarían datos de mercado reales como en el estudio
class StockDataset(Dataset):
    def __init__(self, data, seq_length, is_train=True):
        self.data = data
        self.seq_length = seq_length
        self.is_train = is_train
        
    def __len__(self):
        return len(self.data) - self.seq_length - 1
        
    def __getitem__(self, idx):
        # Secuencia de entrada
        x = self.data[idx:idx+self.seq_length].reshape(-1, INPUT_DIM)
        
        # Valor siguiente (a predecir)
        next_value = self.data[idx+self.seq_length]
        
        # Convertir a etiqueta categórica (0: abajo, 1: plano, 2: arriba)
        # Simplificación - en la práctica se usaría un umbral para determinar "plano"
        current_value = self.data[idx+self.seq_length-1]
        if next_value > current_value * 1.001:  # 0.1% de aumento
            y = 2  # Arriba
        elif next_value  best_val_acc:
            best_val_acc = val_epoch_acc
            torch.save(model.state_dict(), 'best_model.pth')
    
    # Visualizar evolución del entrenamiento
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(train_accs, label='Train Accuracy')
    plt.plot(val_accs, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    return model

### Configuración Experimental y Preprocesamiento de Datos

El modelo CTTS fue evaluado con datos intradía del índice S&P 500[1]. Específicamente, el conjunto de datos constaba de precios de acciones con intervalos de un minuto para todas las empresas constituyentes del S&P 500 durante el año 2019[1]. Las primeras 39 semanas de 2019 se utilizaron para entrenamiento (con una porción para validación), y las 13 semanas restantes (el último trimestre de 2019) sirvieron como conjunto de prueba[1].

Para cada acción, la serie temporal de precios históricos se segmentó en muestras de 80 pasos temporales (80 minutos) como entradas del modelo, con el objetivo de predecir el movimiento del precio en el minuto 81[1]. Esta formulación traduce la tarea de pronóstico en un problema de clasificación: determinar si el precio subirá, bajará o permanecerá sin cambios en el siguiente minuto, dada la historia reciente de 80 minutos.


In [ ]:
# Código para preprocesamiento de datos financieros
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def preprocess_financial_data(file_path, sequence_length=80):
    """
    Preprocesa datos financieros para entrenamiento del modelo CTTS
    
    Args:
        file_path: Ruta al archivo CSV con datos de precios
        sequence_length: Longitud de la secuencia de entrada
    
    Returns:
        X_train, y_train, X_val, y_val, X_test, y_test: Datos preparados
    """
    # Cargar datos
    df = pd.read_csv(file_path)
    
    # Asegurar que los datos estén ordenados por fecha
    if 'date' in df.columns:
        df = df.sort_values('date')
    
    # Seleccionar la columna de precio de cierre
    close_prices = df['close'].values.reshape(-1, 1)
    
    # Escalar los datos al rango [0,1]
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_prices = scaler.fit_transform(close_prices)
    
    # Crear etiquetas (0: abajo, 1: plano, 2: arriba)
    price_diff = np.diff(scaled_prices.flatten())
    # Definir umbrales para "plano" (por ejemplo, cambio menor a 0.001)
    threshold = 0.001
    labels = np.zeros(len(price_diff), dtype=int)
    labels[price_diff > threshold] = 2  # Arriba
    labels[np.abs(price_diff)  threshold:
            pred_classes[i] = 2  # Arriba
        elif diff  threshold:
            actual_classes[i] = 2  # Arriba
        elif diff = threshold:
                    # Si la predicción es "subida" (2) y no tenemos posición
                    if pred == 2 and position == 0:
                        # Comprar
                        shares = capital / 100  # Simplificación - precio = 100
                        capital = 0
                        position = 1
                        transaction_log.append({
                            'action': 'BUY',
                            'shares': shares,
                            'confidence': prob
                        })
                    
                    # Si la predicción es "bajada" (0) o "plano" (1) y tenemos posición
                    elif (pred == 0 or pred == 1) and position == 1:
                        # Vender
                        capital = shares * 100  # Simplificación - precio = 100
                        position = 0
                        transaction_log.append({
                            'action': 'SELL',
                            'shares': shares,
                            'capital': capital,
                            'confidence': prob
                        })
    
    # Calcular rendimiento final
    if position == 1:
        # Vender cualquier posición restante al final
        capital = shares * 100
    
    performance = (capital - initial_capital) / initial_capital * 100
    
    print(f"Capital inicial: ${initial_capital:.2f}")
    print(f"Capital final: ${capital:.2f}")
    print(f"Rendimiento: {performance:.2f}%")
    print(f"Número de transacciones: {len(transaction_log)}")
    
    return performance, transaction_log

## Conclusiones y Direcciones Futuras

El modelo CNN-Transformer para Series Temporales (CTTS) propuesto por Zeng et al. representa un avance significativo en el pronóstico de series temporales financieras, demostrando la eficacia de combinar CNNs para la extracción de características locales con Transformers para capturar dependencias a largo plazo. Los resultados experimentales confirman que este enfoque híbrido supera tanto a los métodos estadísticos tradicionales como a otros modelos de aprendizaje profundo en la predicción de movimientos de precios de acciones.

La investigación demuestra que las CNNs, tradicionalmente asociadas con el procesamiento de imágenes, pueden adaptarse efectivamente para analizar patrones en datos secuenciales, especialmente cuando se combinan con arquitecturas especializadas en la captura de dependencias a larga distancia. Esta flexibilidad abre nuevas posibilidades para aplicaciones en diversos dominios donde existen series temporales complejas con patrones a múltiples escalas.

Entre las direcciones futuras para este campo se incluyen:

1. **Incorporación de datos multimodales**: Integrar información textual (como noticias financieras o sentimiento en redes sociales) junto con datos de precios para mejorar aún más la precisión predictiva.

2. **Arquitecturas más complejas**: Explorar variaciones arquitectónicas como Temporal Convolutional Networks (TCN) combinadas con variantes de Transformer más eficientes.

3. **Expansión a horizontes de predicción más largos**: Adaptar el modelo para predecir movimientos de precios a plazos más largos (días o semanas), lo que requeriría estrategias para manejar la creciente incertidumbre.

4. **Aplicación a otros dominios**: Extender el enfoque CNN-Transformer a otros campos como pronóstico de demanda, predicción meteorológica, o monitoreo de salud, donde también existen patrones temporales complejos.

5. **Interpretabilidad del modelo**: Desarrollar técnicas para visualizar y comprender qué patrones específicos están detectando las CNNs y a qué relaciones temporales están prestando atención los Transformers, lo que podría proporcionar insights valiosos para analistas y operadores.

En conclusión, el modelo CNN-Transformer demuestra el potencial de los enfoques híbridos de aprendizaje profundo para abordar problemas complejos de pronóstico de series temporales. Su capacidad para capturar simultáneamente patrones a corto y largo plazo ofrece una ventaja significativa sobre los métodos tradicionales, con aplicaciones prácticas prometedoras en finanzas y más allá.


# Citations:
• [1] https://ppl-ai-file-upload.s3.amazonaws.com/web/direct-files/50879975/c2506f19-550e-43f8-a03e-324bd56112db/Articulo_json.txt
• [2] https://arxiv.org/abs/2304.04912
• [3] https://rcs.cic.ipn.mx/2022_151_7/Prediccion%20de%20senales%20financieras%20usando%20una%20red%20neuronal%20convolucional%20de%20memoria%20de%20corto%20plazo.pdf
• [4] https://riunet.upv.es/bitstream/handle/10251/164734/(1)_Martinez%20-%20Predicci%C3%B3n%20de%20demanda%20y%20producci%C3%B3n%20de%20energ%C3%ADa%20el%C3%A9ctrica%20mediante%20redes%20neuronales%20y%20va....pdf?sequence=2
• [5] https://es.linkedin.com/posts/naren-castellon-1541b8101_machinelearning-arima-lstm-activity-7198107050055204864-fSCk
• [6] https://fastercapital.com/es/tema/%C2%BFcu%C3%A1les-son-las-ventajas-de-utilizar-redes-neuronales-convolucionales-(cnn)-en-el-modelado-de-secuencias.html
• [7] https://arxiv.org/pdf/2304.04912.pdf
• [8] https://es.linkedin.com/pulse/tiempedia-modelos-cnn-aplicados-series-de-tiempo-mora-caballero-lwmne
• [9] https://cnnespanol.cnn.com/2023/06/15/aumentan-previsiones-calor-extremo-texas-mexico-cambio-climatico-trax
• [10] https://ligeia.eafit.edu.co/bitstreams/7af18199-22ef-4a6a-b3c9-b4fb01097f49/download
• [11] https://www.isdi.education/es/?p=26288
• [12] https://blogs.upm.es/pasd/2024/06/06/analisis-de-series-temporales-en-finanzas-con-inteligencia-artificial-2/
• [13] https://www.cnn.com/videos/spanish/2022/09/14/cortes-energia-infraestructura-elctrica-cambio-climatico-temperaturas-eeuu-dusa-roa.cnn
• [14] http://www.scielo.org.co/scielo.php?script=sci_arttext&pid=S0123-30332023000300014
• [15] https://dspace.udla.edu.ec/handle/33000/15636
• [16] https://migueljimenezg.github.io/cursos/Machine%20Learning/Deep%20Learning/CNN-LSTM%20para%20series%20de%20tiempo/CNN-LSTM%20para%20series%20de%20tiempo.html
• [17] https://oa.upm.es/82673/1/TFG_GUILLERMO_FILGUEIRA_FERNANDEZ.pdf
• [18] https://repositori.uji.es/xmlui/bitstream/handle/10234/201358/TFG_2022_Peris_Tena_Francisco.pdf?sequence=1
• [19] https://pro.arcgis.com/es/pro-app/latest/tool-reference/geoai/how-time-series-forecasting-models-work.htm
• [20] https://dialnet.unirioja.es/descarga/articulo/9039551.pdf
• [21] https://cnnespanol.cnn.com/2023/12/24/noticias-sombrias-clima-5-razones-optimistas-trax
